<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [ ]:
# Data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# Plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Stats/regressions packages
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV

# Make sure your charts appear in the notebook
%matplotlib inline
%config InlineBackend.figure_format ='retina'



In [ ]:
import warnings
warnings.filterwarnings('ignore')  # sorry for that, really

import datetime as d_t

def display_df(df):
    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
        display(df)

def list_df(list1d, maxcols=3):
    if type(list1d)=='list':
        newlist = list1d
    else:
        newlist = list(list1d)
    n = len(newlist) % maxcols
    n = 0 if n==0 else (maxcols -n)
    if n > 0:
        newlist = newlist + [''] * n 
    df=pd.DataFrame( {' '+str(n) : [v for x,v in enumerate(newlist) if x % maxcols == n] for n in range(maxcols)} )
    return df

def overview(df, keycol=''): 
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nuniq = list(df.nunique())
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    skewness = df.skew()
    kurtosis = df.kurt() 
    print('Data shape:', df.shape)

    dt = df.dtypes.to_frame()
    dtitle = 'Data type'
    dt.columns = [dtitle]
    print( dt.groupby(dtitle)[dtitle].count() )
    
    if keycol == '':
        cols = ['types', 'counts', 'distincts', 'nulls', 'uniques', 'skewness', 'kurtosis']
        df_overview = pd.concat([types, counts, distincts, nulls,  uniques, skewness, kurtosis], axis = 1)
    else:
        corr = df.corr()
        cols = ['types', 'counts', 'distincts', 'nulls', 'uniques', 'skewness', 'kurtosis', 'corr' ]
        df_overview = pd.concat([types, counts, distincts, nulls, uniques, skewness, kurtosis, corr[keycol]], axis = 1)
        
    df_overview.columns = cols
    df.columns.name="Cols"
    df.index.name=""
    print('\n')
    return df_overview

def dtype_by(df, datatype):
    return df.select_dtypes(include=datatype).T.index

In [ ]:
# Load the data
house = pd.read_csv('./housing.csv')

## Analyzing the data

In [ ]:
display_df(house.head())

In [ ]:
house.drop(['Id'], axis=1, inplace=True)

In [ ]:
# checking the columns
list_df(house.columns,5)

In [ ]:
# not so clean data
house_info = overview(house)

# prepare to fix the null values, lookup for the top counts
list_null = house_info[house_info.nulls>0].sort_values('nulls',ascending=False).nulls

house_info

In [ ]:
list_df( dtype_by(house,'object') ,5)

In [ ]:
# highest 755,000
house['SalePrice'].describe()

In [ ]:
# 140,000 high frequency
house['SalePrice'].value_counts().head()

In [ ]:
house.head()

## Data Processing

In [ ]:
# SalePrice is not uniformly distributed and is skewed towards the left .
# Quick glance, take this variable to be our objective
house['SalePrice'].hist(bins = 40)

In [ ]:
# Use log1p to remove the skewness 
house['SalePrice'] = np.log(house['SalePrice'])
status = house['SalePrice'].reset_index(drop=True)

In [ ]:
# The distribution of SalesPrice is more balanced 
# Quick glance, take this variable to be our objective
house['SalePrice'].hist(bins = 40)

In [ ]:
# to fix these columns having missing values
list_null

In [ ]:
#after running thru each columns via .unique and listing, basic fixing in action 
#LotFrontage
house['LotFrontage'] = house['LotFrontage'].fillna(0)
#Alley
house['Alley'] = house['Alley'].fillna('None')
#MasVnrType
house['MasVnrType'] = house['MasVnrType'].fillna('None')
#MasVnrArea
house['MasVnrArea'] = house['MasVnrArea'].fillna(0)
#BsmtQual
house['BsmtQual'] = house['BsmtQual'].fillna('None')
#BsmtCond
house['BsmtCond'] = house['BsmtCond'].fillna('None')
#BsmtExposure
house['BsmtExposure'] = house['BsmtExposure'].fillna('None')
#BsmtFinType1
house['BsmtFinType1'] = house['BsmtFinType1'].fillna('None')
#BsmtFinType2
house['BsmtFinType2'] = house['BsmtFinType2'].fillna('None')
#Electrical
house['Electrical'] = house['Electrical'].fillna('None')
#FireplaceQu
house['FireplaceQu'] = house['FireplaceQu'].fillna('None')
#GarageType
house['GarageType'] = house['GarageType'].fillna('None')
#GarageYrBlt
house['GarageYrBlt'] = house['GarageYrBlt'].fillna(0)
#GarageFinish
house['GarageFinish'] = house['GarageFinish'].fillna(0)
#GarageQual
house['GarageQual'] = house['GarageQual'].fillna('None')
#GarageCond
house['GarageCond'] = house['GarageCond'].fillna('None')
#PoolQC
house['PoolQC'] = house['PoolQC'].fillna('None')
#Fence
house['Fence'] = house['Fence'].fillna('None')
#MiscFeature
house['MiscFeature'] = house['MiscFeature'].fillna('None')

In [ ]:
#no more missing values
house.isnull().sum().sum()

In [ ]:
# list of variables related to "square feet"
# run below to take a look
! grep -i 'square feet' ./data_description.txt

In [ ]:
# Since those area related columns having definition with square feet , should be float
list_col = ! grep -i 'square feet' data_description.txt | awk -F: '{ print $1}' | grep -v MasVnrArea
print("changing data type from int64 to float for :")
for col in list_col:
    print(col)
    house[col] = house[col].astype(float)    

In [ ]:
# last look , the rest were correctly integer type
for x in house.select_dtypes(include='int64').dtypes.items():
    col=x[0]
    ! grep -i $col data_description.txt

# list out those columns to check again    
house[ dtype_by(house,'int64') ].head()    

In [ ]:
# Could these be float instead of integer ? Such as 'Area' related ?
list_df( dtype_by(house,'int64') ,5)

In [ ]:
house[ dtype_by(house,'int64') ].head()

In [ ]:
# Find float64 that can be in the continuous variables 
list_df( dtype_by(house,'float') ,5)

In [ ]:
# one of above should be integer for keeping years value.
house.GarageYrBlt = house.GarageYrBlt.apply(lambda x: int(x) )
house['GarageYrBlt'] = house['GarageYrBlt'].astype(int)

In [ ]:
# checked , no need to convert anything to integer type
house[ dtype_by(house,'float') ].head()

In [ ]:
# these will be out of training data
list_yrs_col=['YearBuilt', 'YrSold', 'YearRemodAdd', 'GarageYrBlt']
house[list_yrs_col][house.YrSold>=2010].head()

In [ ]:
house[list_yrs_col][house.YearBuilt>=2010]

In [ ]:
house[list_yrs_col][house.YearRemodAdd>=2010]

In [ ]:
house[list_yrs_col][house.GarageYrBlt>=2010]

In [ ]:
print(house.YearBuilt.unique())
print(house.YrSold.unique())
print(house.YearRemodAdd.unique())
print(house.GarageYrBlt.unique())

In [ ]:
# make it number of years by now, later on will rename the columns
year_now = d_t.datetime.now().year
house['Yrs_Sold'] = house.eval(str(year_now) + ' - YrSold')
house['Yrs_Built'] = house.eval(str(year_now) + ' - YearBuilt')
house['Yrs_Remodel'] = house.eval(str(year_now) + ' - YearRemodAdd')
house['Yrs_GarageBlt'] = house.eval(str(year_now) + ' - GarageYrBlt')

In [ ]:
list_yrs_col=['YearBuilt', 'YrSold', 'YearRemodAdd', 'GarageYrBlt', \
'Yrs_Built', 'Yrs_Remodel' , 'Yrs_GarageBlt' , 'Yrs_Sold']

house[list_yrs_col].head()

In [ ]:
# rename columns to make it readable if possible

house.columns = \
["BuildingSalestype", "ZoningClassification", "StreetDistance", "LotSize", "Street", "Alley", \
"GeneralShape", "Flatness", "Utilities", "LotConfig", "Slope", "Neighborhood", "Condition1", \
"Condition2", "BuildingType", "HouseStyle", "OverallQuality", "RatingCondition", "YearBuilt", \
"YearRemodAdd", "RoofStyle", "RoofMaterial", "Exterior1st", "Exterior2nd", "MasonryType", "MasonryArea", \
"ExteriorQuality", "ExterCond", "Foundation", "BasementHeight", "BasementCondition", "BasementExposure",  \
"RatingBasement1", "BasementArea1", "RatingBasement2", "BasementArea2", "BasementUnfinished", "TotalBasementArea",  \
"Heating", "HeatingQC", "CentralAir", "Electrical", "FirstFloor", "SecondFloor", \
"LowQualFinSF", "AreaAboveGround", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", \
"KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType", \
"GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual", "GarageCond", "PavedDrive", \
"WoodDeckArea", "OpenPorchArea", "EnclosedPorchArea", "SeasonPorchArea", "ScreenPorch", "PoolArea", \
"PoolQC", "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold", "SaleType", "SaleCondition", "SalePrice", \
 "Yrs_Built", "Yrs_Sold", "Yrs_Remodel", "Yrs_GarageBlt"]

house.head()

In [ ]:
# remove NA for remaining columns if I still missed 
house.dropna(inplace=True)

## Data Quality

In [ ]:
house.drop(house[house.ZoningClassification=='C (all)'].index, inplace=True)

In [ ]:
house.shape

In [ ]:
house.groupby('ZoningClassification')['ZoningClassification'].head(1)

In [ ]:
# now clean data, run again
house_info = overview(house , "SalePrice")
house_info

## Correlation Matrix

In [ ]:
house_info = overview(house , "SalePrice")

# looking correlation value to SalesPrice
house_info.sort_values(by='corr',ascending=False)['corr'][:25]

In [ ]:
fig = plt.figure(figsize=(20, 15))
sns.set(font_scale=1.5)

fig1 = fig.add_subplot(221); 
sns.boxplot(x='OverallQuality', y='SalePrice', data=house[['SalePrice', 'OverallQuality']])
fig2 = fig.add_subplot(222); 
sns.scatterplot(x = house.AreaAboveGround, y = house.SalePrice, hue=house.OverallQuality, palette= 'Spectral')
fig3 = fig.add_subplot(223); 
sns.scatterplot(x = house.GarageCars, y = house.SalePrice, hue=house.OverallQuality, palette= 'Spectral')
fig4 = fig.add_subplot(224); 
sns.scatterplot(x = house.GarageArea, y = house.SalePrice, hue=house.OverallQuality, palette= 'Spectral')


plt.title("Overall Quality")
plt.tight_layout(); plt.show()


In [ ]:
fig = plt.figure(figsize=(20, 15))
sns.set(font_scale=1.5)

fig5 = plt.figure(figsize=(16, 8))
fig6 = fig5.add_subplot(121); 
sns.scatterplot(y = house.SalePrice , x = house.TotalBasementArea, hue=house.OverallQuality, palette= 'YlOrRd')
fig7 = fig5.add_subplot(122); 
sns.scatterplot(y = house.SalePrice, x = house['FirstFloor'], hue=house.OverallQuality, palette= 'YlOrRd')
fig8 = plt.figure(figsize=(16, 8))
fig9 = fig8.add_subplot(121); 
sns.boxplot(x='FullBath', y='SalePrice', data=house[['SalePrice', 'FullBath']])
fig10 = fig8.add_subplot(122); 
sns.boxplot(x='TotRmsAbvGrd', y='SalePrice', data=house[['SalePrice', 'TotRmsAbvGrd']])

plt.title("Overall Quality")
plt.tight_layout(); plt.show()


In [ ]:
fig = plt.figure(figsize=(20,5))
ax = fig.add_subplot(121)
sns.set()
sns.scatterplot(x = house.AreaAboveGround, y = house.SalePrice, ax = ax)
plt.annotate('Outlier', xy=(4000, 10), xytext=(5642, 11.9), arrowprops=dict(facecolor='g', shrink=0.05)    )
plt.annotate('Outlier', xy=(4000, 10), xytext=(4676, 12.1), arrowprops=dict(facecolor='g', shrink=0.05)    )


# calculate outliers
outlier_high_x = np.mean(house.AreaAboveGround) + np.std(house.AreaAboveGround) * 1.5
outlier_low_y = np.mean(house.SalePrice) - np.std(house.SalePrice) * 1.5

print(outlier_high_x,outlier_low_y)
mask = (house.AreaAboveGround >= outlier_high_x) & (house.SalePrice <= outlier_low_y )
#Deleting outliers
idx=house.loc[mask].index[0]
house.drop( house.index[idx] )
mask = (house.AreaAboveGround >= outlier_high_x) & (house.SalePrice <= outlier_low_y )
idx=house.loc[mask].index[0]
house.drop( house.index[idx], inplace=True )
house.drop( house[(house.AreaAboveGround >=4000) & (house.SalePrice <=300000)].index, inplace=True)

#Check the graphic again
ax = fig.add_subplot(122)
sns.scatterplot(x = house.AreaAboveGround, y = house.SalePrice, ax = ax)
plt.show()

In [ ]:
# more to add as features
features = [ \
"OverallQual", "GrLivArea", "GarageCars", "GarageArea", 
"TotalBsmtSF", "FirstFloor", "FullBath", "TotRmsAbvGrd", \
"YearBuilt", "YearRemodAdd", "GarageYrBlt", "MasVnrArea", \
"Fireplaces", "BsmtFinSF1", "LotFrontage", "WoodDeckSF", \
"SecondFloor", "OpenPorchSF", "HalfBath", "LotArea", \
"BsmtFullBath", "BsmtUnfSF", "BedroomAbvGr", "ScreenPorch" ]

## Features Engineering

In [ ]:
# Mapping ordinals features
house_info = overview(house , "SalePrice")

# dictionary of list for columns with unique values <= 10
cols_uniq = house_info[house_info.distincts<=10].uniques
print(cols_uniq)


In [ ]:
print('List of columns were categorial can be mapped to digital category')
# next, prepare a list of categorial columns 
categorial_cols = []
dict_categories={}
for c in cols_uniq.items():
    key=c[0]
    keylist=c[1][0]
    elem=c[1][0][0]   
    if len([c for c in str(elem) if c not in '0123456789.'])>0:
        dict_categories[key]=keylist
        categorial_cols.append(key)

print("# of columns : ",len(categorial_cols))
list_df(categorial_cols, 4)

In [ ]:
dummydf = pd.get_dummies(house[categorial_cols], columns = categorial_cols)

In [ ]:
house = pd.concat([house, dummydf], axis=1)

In [ ]:
for col in house.columns:
    newcol = col.replace('.','').replace('&','')
    if col != newcol:
        print(col, newcol)
        house.rename(columns={col:newcol}, inplace=True)

In [ ]:
for col in categorial_cols:
    house.drop([col], axis=1, inplace=True)

In [ ]:
# 249 columns
house.shape

In [ ]:
display_df(house.head())

## Prepare Training Data

In [ ]:
trainmask = (house.YearBuilt < 2010) & (house.Yrs_Remodel < 2010) & \
    (house.Yrs_GarageBlt < 2010) & (house.Yrs_Sold < 2010)

In [ ]:
list_yrs_col=['YearBuilt', 'YrSold', 'YearRemodAdd', 'GarageYrBlt', \
'Yrs_Built', 'Yrs_Remodel' , 'Yrs_GarageBlt' , 'Yrs_Sold']

In [ ]:
house[list_yrs_col][trainmask].count()

In [ ]:
house_train = house[trainmask]
house_holdout = house[~trainmask]

In [ ]:
house.shape, house_train.shape, house_holdout.shape

In [ ]:
house_train[list_yrs_col].head()

In [ ]:
# backup all the hardwork to save time
house.to_csv('house_clean.csv')
house_train.to_csv('house_train.csv')
house_holdout.to_csv('house_holdout.csv')

In [ ]:
#house = pd.read_csv('house_clean.csv', index_col=[0])
#house_train = pd.read_csv('house_train.csv', index_col=[0])

## Predictor and Target

In [ ]:
target = 'SalePrice'
house_info = overview(house_train , target)
list_cols = [col for col in house_train.columns if col != target]
X = house_train[list_cols]
y = house_train[target]

In [ ]:
# looking correlation value to SalesPrice
house_info.sort_values(by='corr',ascending=False)['corr'][:25]

In [ ]:
features = ["OverallQuality", "AreaAboveGround", "GarageCars", "TotalBasementArea", \
"GarageArea", "FirstFloor", "FullBath", "YearBuilt", "YearRemodAdd", \
"GarageYrBlt", "TotRmsAbvGrd", "Foundation_PConc", "ExteriorQuality_Gd", \
"BasementHeight_Ex", "Fireplaces", "HeatingQC_Ex", "RatingBasement1_GLQ", \
"KitchenQual_Ex", "MasonryArea", "GarageFinish_Fin", "KitchenQual_Gd", \
"ExteriorQuality_Ex", "OpenPorchArea", "BasementArea1"]

In [ ]:
display_df(house_train[:10])

In [ ]:
cols_to_skip = ['SalePrice', 'Neighborhood','Exterior1st','Exterior2nd', 'YearBuilt','YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [ ]:
list_cols = [ x for x in house.columns if x not in cols_to_skip]

In [ ]:
status = house_train.reset_index(drop=True)
house_train.head()

In [ ]:
#status = house_train.reset_index(drop=True)
#house_train.head()

## Modelling

In [ ]:
X = house_train[list_cols]
y = house_train[target]

In [ ]:
# splt training data
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
print(X.shape, xTrain.shape, xTest.shape, house_train.shape)

## Lasso , Ridge , ElasticNet

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

linreg = LinearRegression()

linreg_scores = cross_val_score(linreg, xTrain, yTrain, cv=10)

print(linreg_scores)
print(np.mean(linreg_scores))

In [ ]:
ridge_alphas = np.logspace(-2, 7, 50)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(xTrain, yTrain)

print(optimal_ridge.alpha_)

In [ ]:
ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, xTrain, yTrain, cv=10)

print(ridge_scores)
print(np.mean(ridge_scores))

In [ ]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(xTrain, yTrain)

print(optimal_lasso.alpha_)

In [ ]:
lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso, xTrain, yTrain, cv=10)

print(lasso_scores)
print(np.mean(lasso_scores))

In [ ]:
lasso.fit(xTrain, yTrain)

In [ ]:
lasso_coefs = pd.DataFrame({'variable':X.columns,
                            'coef':lasso.coef_,
                            'abs_coef':np.abs(lasso.coef_)})

lasso_coefs.sort_values('abs_coef', inplace=True, ascending=False)

lasso_coefs.head(20)

In [ ]:
print('Percent variables zeroed out:', len(lasso_coefs[lasso_coefs['coef'] == 0])/float(len(lasso_coefs)))

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)

optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, 
                            n_alphas=30, 
                            cv=10, 
                            max_iter=10000)
optimal_enet.fit(xTrain, yTrain)

print(optimal_enet.alpha_)
print(optimal_enet.l1_ratio_)

In [ ]:
enet = ElasticNet(alpha=optimal_enet.alpha_, 
                  l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, xTrain, yTrain, cv=10)

print(enet_scores)
print(np.mean(enet_scores))

In [ ]:
# fit the elastic net and ridge outside of cross_val_score
ridge.fit(xTrain, xTrain)
lasso.fit(xTrain, yTrain)

## OLS Model

In [ ]:
import statsmodels.formula.api as sm

ln_model = sm.OLS(yTrain,xTrain)
result = ln_model.fit()
#print(result.summary2())
result.summary()

In [ ]:
print("top 50 best P-Values")
list_p1 = [str(v)+'_'+x for x,v in result.pvalues.items() if v==v]
sorted(list_p1,reverse=True)[:10]

# exclude 'Utilities_NoSeWa', 'RoofMaterial_Membran'

In [ ]:
print(f"# of features = {len(result.pvalues)}")
print(f"R-Square =  {result.rsquared}")

In [ ]:
sorted(list(ln_model.score(result.params)), reverse=True)[:5]

In [ ]:
list_p2 = [str(v)+'_'+x for x,v in result.pvalues.items() if v<0.5]
print(f"P-values below 0.5 , count = {len([str(v)+'_'+x for x,v in result.pvalues.items() if v<0.5])}")
list_p2[:10]

In [ ]:
# eliminate some columns, I gpt 93 columns removed
def backwardElimination(x, Y, sl, columns):
    ini = len(columns)
    numVars = x.shape[1]
    for i in range(0, numVars):
        regressor = sm.OLS(Y, x).fit()
        maxVar = max(regressor.pvalues) #.astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor.pvalues[j].astype(float) == maxVar):
                    columns = np.delete(columns, j)
                    x = x.loc[:, columns]
                    
    print('\nSelect {:d} features from {:d} by best p-values.'.format(len(columns), ini))
    print('The max p-value from the features selecte is {:.3f}.'.format(maxVar))
    print(regressor.summary())
    
    # odds ratios and 95% CI
    conf = np.exp(regressor.conf_int())
    conf['Odds Ratios'] = np.exp(regressor.params)
    conf.columns = ['2.5%', '97.5%', 'Odds Ratios']
    display(conf)
    
    return columns, regressor

SL = 0.051
pv_cols = list_cols
pv_cols, LR = backwardElimination(xTrain, yTrain, SL, pv_cols)


In [ ]:
# exclude 'Utilities_NoSeWa', 'RoofMaterial_Membran'
for col in ['Utilities_NoSeWa', 'RoofMaterial_Membran']:
    n=list(pv_cols).index(col)
    np.delete(pv_cols,n)

smols_columns = pv_cols

In [ ]:
list_df( pv_cols ,5)

## RFECV Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso

ls = Lasso(alpha = 0.0005, max_iter = 161, selection = 'cyclic', tol = 0.002, random_state = 101)
rfecv = RFECV(estimator=ls, n_jobs = -1, step=1, scoring = 'neg_mean_squared_error' ,cv=5)
selector = rfecv.fit(xTrain, yTrain)

#select_features_rfecv = rfecv.get_support()
select_features_rfecv=selector.support_
#print(selector.support_)
print(selector.ranking_)

In [ ]:
# The column names correspond to the ones below. RFECV only excluded a few features.
rfecv_columns = np.array(list_cols)[selector.support_]

In [ ]:
print("eliminated some columns, left with:",rfecv_columns.shape)

In [ ]:
selector

In [ ]:
house_train[rfecv_columns].head()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# Build the selector — we'll build one with each score type.
skb_f = SelectKBest(f_classif, k=5)
skb_chi2 = SelectKBest(chi2, k=5)

# Train the selector on the data.
skb_f.fit(xTrain, yTrain)

try:
    skb_chi2.fit(xTrain, yTrain)
    # Examine the results.
    kbest = pd.DataFrame([list_cols, list(skb_f.scores_), list(skb_chi2.scores_)], 
                         index=['feature','f_classif','chi2 score']).T.sort_values('f_classif', ascending=False)
    
except:
    # print("skip this debugging, no time to troubleshoot")
    kbest = pd.DataFrame([list_cols, list(skb_f.scores_)], 
                         index=['feature','f_classif']).T.sort_values('f_classif', ascending=False)

kbest_columns=kbest.feature.values
kbest_scores=kbest[kbest.f_classif <= 50].f_classif
kbest[kbest.f_classif <= 50][:10]

In [ ]:
kbest_columns[0:5]

In [ ]:
kbest_scores[:5]

## Model Selection

In [ ]:
optimal_enet.score(xTest, yTest)

In [ ]:
optimal_ridge.score(xTest, yTest)

In [ ]:
optimal_lasso.score(xTest, yTest)

In [ ]:
## OLS model prediction not in use

## Model Selected

## Holdout Test


In [ ]:
target = 'SalePrice'
house_info = overview(house_holdout , target)
list_cols = [col for col in house_holdout.columns if col != target]
X = house_holdout[xTrain.columns].values
y = house_holdout[target].values

In [ ]:
house_holdout.shape, house_train.shape, X.shape, xTrain.shape

In [ ]:
try:
    Ypred = optimal_ridge.predict(X)
except:
    print('unable to work the predictions. failure')

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
score = mean_absolute_error(y_pred=np.expm1(Ypred), y_true=np.expm1(y))
score

In [ ]:
print(f"the best estimates of price is ${score}")

In [ ]:
Ypred = optimal_lasso.predict(X)
score = mean_absolute_error(y_pred=np.expm1(Ypred), y_true=np.expm1(y))
score

In [ ]:
Ypred = optimal_enet.predict(X)
score = mean_absolute_error(y_pred=np.expm1(Ypred), y_true=np.expm1(y))
score

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# sorry , didn't do in time